In [119]:
# import all libraries
from plotly import graph_objects as go
import plotly.express as px
import pandas as pd
import dash
from dash.dependencies import Input, Output
from dash import Dash, html, dcc 
from urllib.request import urlopen
import json
from plotly.subplots import make_subplots

In [120]:
# Load Veggie Data
df = pd.read_csv("kv21_trimmet_98.csv",
                 dtype={"fips": str})
df = df.fillna(0) # replace NA values with 0
df_nameIndex = df.set_index("Navn")

In [146]:
# Input til graferne
kommune = df_nameIndex["Kommune"].unique()[0]
df_temp =  df_nameIndex[df_nameIndex["Kommune"]==kommune]
question = df_nameIndex.columns[4]
color_dict = { "Enig":'rgb(15,122,55)',"Delvist Enig": 'rgb(169,220,163)',"Uenig":'rgb(218,241,212)'}
print(kommune, question)
colors = ['rgb(218,241,212)','rgb(169,220,163)','rgb(15,122,55)']

Albertslund 1. "Kommunen skal prioritere, at skolerne får flere midler til indkøb af klimavenlige og bæredygtige råvarer til madkundskab"


In [147]:
kommune = df_nameIndex["Kommune"].unique()[0]
kommune

'Albertslund'

## Lollipop 4 eva

In [148]:
def Marker_size(lenght):
    markerList = []
    for i in range(lenght):
        markerList.extend([0,10])
    return markerList

In [149]:


for i, name in enumerate(df_temp.index):
    print(i, name,df_temp.loc[name]["Score"])

0 Flemming Jørgensen 9
1 Nicolai Kofod-Jensen 16
2 Billal Zahoor 16
3 Inge Dahn 23
4 Michael Jonasen 16
5 Adam Samy 32
6 Jeppe Milthers 18
7 Lars Bremer 18


In [150]:
fig = go.Figure()

df_temp = df_nameIndex[df_nameIndex["Kommune"]==kommune].sort_values("Score", ascending = False)

tickvals_ = list(range(len(df_temp)))

ticktext_ = list(df_temp.index)


veggieGreen = 'rgb(5,122,87)' # Dark-green for the vegetarian color option !!! Change for real color
markerSize = Marker_size(len(df_temp))


for i, mean in enumerate(df_temp["Score"]):
    candidate = df_temp.index[i]
    fig.add_trace(go.Scatter(x=[i,i],y=[0,mean],
                             marker={"color":veggieGreen,"size":markerSize},
                             line=go.scatter.Line(color=veggieGreen),
                             hovertext=[df_temp.loc[candidate]["Parti"],df_temp.loc[candidate]["Parti"]],
                             showlegend=False))
    
fig.add_hline(y=df_temp["Score"].mean(), 
              line_width=0.5, 
              line_dash="dash", 
              line_color=veggieGreen,
              annotation_text="Kommune gennemsnit", 
              annotation_position="bottom right")

fig.update_layout(
    xaxis = dict(
        tickmode = "array",
        tickvals = tickvals_,
        ticktext = ticktext_),
    xaxis_range=[-1,len(df_temp)],
    title = {"text":f"Kandidater for {kommune}"}
    )

    
fig.show()

# Pie-chart

In [151]:
# df_temp =  df_nameIndex[df_nameIndex["Kommune"]==df_nameIndex["Kommune"][10]]
# answer_labels = {2:"Enig", 1:"Delvist Enig",0: "Uenig"}
# df_temp["question_labels"] = [answer_labels[x] for x in df_temp[df_temp.columns[5]]]


df_pie = df_temp.groupby(df_temp.columns[5]).count()
df_pie2 = df_temp.groupby(df_temp.columns[6]).count()


# labels = ["Enig", "Delvist Enig", "Uenig"]
# df_pie.reindex(labels)["Score"]

In [217]:
df_temp =  df_nameIndex[df_nameIndex["Kommune"]==kommune]
df_temp = df_temp.sort_values(question, ascending = False)


# labels for piechart - kommune
value_labels = {0:"Uenig", 1:"Delvist Enig", 2:"Enig"}
answers_kommune = pd.Series([value_labels[x] for x in df_temp[question]], 
                              index =df_temp.index)
labels_kommune = list(answers_kommune.unique())

# Value for kommune
df_pie = df_temp.groupby(df_temp[question]).count()
value_kommune = df_pie["Score"]

# Colors kommune
color_dict = { "Enig":'rgb(15,122,55)',"Delvist Enig": 'rgb(169,220,163)',"Uenig":'rgb(218,241,212)'}
colors_pie = {}
for answer in labels_kommune:
    colors_pie[answer] = color_dict[answer]

fig = go.Figure()

fig.add_trace(go.Pie(labels=labels_kommune, 
                            values=value_kommune, 
                            hole=0.6,
                            marker_colors = list(colors_pie.values()),
                            ))


In [220]:
df_temp =  df_nameIndex[df_nameIndex["Kommune"]==kommune]
df_temp = df_temp.sort_values(question, ascending = False)


fig_pie = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])

# labels for piechart - kommune
value_labels = {0:"Uenig", 1:"Delvist Enig", 2:"Enig"}
answers_kommune = pd.Series([value_labels[x] for x in df_temp[question]], 
                              index =df_temp.index)
labels_kommune = list(answers_kommune.unique())

# Value for kommune
df_pie = df_temp.groupby(df_temp[question]).count()
value_kommune = df_pie["Score"]

# Colors kommune
color_dict = { "Enig":'rgb(15,122,55)',"Delvist Enig": 'rgb(169,220,163)',"Uenig":'rgb(218,241,212)'}
colors_pie = {}
for answer in labels_kommune:
    colors_pie[answer] = color_dict[answer]


fig_pie.add_trace(go.Pie(labels=labels_kommune, 
                            values=value_kommune, 
                            hole=0.6,
                            marker_colors = list(colors_pie.values()),
                            ),row = 1,col = 1)


df_pie2 = df_nameIndex.groupby(df_nameIndex[question]).count()
values = df_pie2["Score"]
labels = ["Uenig", "Delvist Enig", "Enig"]
colors = ['rgb(218,241,212)','rgb(169,220,163)','rgb(15,122,55)']


fig_pie.add_trace(go.Pie(labels=labels, 
                            values=values, 
                            hole=0.6,
                            marker_colors = colors
                            ),row = 1,col = 2)


fig_pie.add_trace(go.Sunburst(
    labels=[kommune],
    parents=[""],
    values=[1],
    ), row = 1, col=1)


fig_pie.add_trace(go.Sunburst(
    labels=["Danmark"],
    parents=[""],
    values=[1],
    ), row = 1, col=2)

fig_pie.update_layout(
    autosize=False,
    width=1200,
    height=720)

# Sunburst chart

In [163]:
# question = df_temp.columns[5]
# municipality = df_nameIndex["Kommune"][10]
# df_temp =  df_nameIndex[df_nameIndex["Kommune"]==df_nameIndex["Kommune"][10]]
# color_dict = { "Enig":'rgb(15,122,55)',"Delvist Enig": 'rgb(169,220,163)',"Uenig":'rgb(218,241,212)'}

In [164]:
df_temp = df_temp.sort_values(question, ascending = False)

In [165]:
print(kommune, question)

Albertslund 1. "Kommunen skal prioritere, at skolerne får flere midler til indkøb af klimavenlige og bæredygtige råvarer til madkundskab"


In [166]:
# Parents for sunburst
value_labels = {0:"Uenig", 1:"Delvist Enig", 2:"Enig"}
parents_candidates = pd.Series([value_labels[x] for x in df_temp[question]], 
                              index =df_temp.index)

sunburst_parents = ["","",""]
sunburst_parents.extend(parents_candidates)
sunburst_parents

['',
 '',
 'Enig',
 'Enig',
 'Enig',
 'Enig',
 'Delvist Enig',
 'Delvist Enig',
 'Delvist Enig',
 'Delvist Enig']

In [167]:
parents_candidates

Navn
Adam Samy                       Enig
Jeppe Milthers                  Enig
Lars Bremer                     Enig
Michael Jonasen                 Enig
Inge Dahn               Delvist Enig
Nicolai Kofod-Jensen    Delvist Enig
Billal Zahoor           Delvist Enig
Flemming Jørgensen      Delvist Enig
dtype: object

In [168]:
# Names for sunburst
sunburst_names = []

inner_names = list(parents_candidates.value_counts(sort=False).index)
sunburst_names.extend(inner_names)

candidate_names = list(df_temp.index)
sunburst_names.extend(candidate_names)

sunburst_names

['Enig',
 'Delvist Enig',
 'Adam Samy',
 'Jeppe Milthers',
 'Lars Bremer',
 'Michael Jonasen',
 'Inge Dahn',
 'Nicolai Kofod-Jensen',
 'Billal Zahoor',
 'Flemming Jørgensen']

In [169]:
# Values for sunburst
sunburst_values = []

inner_values = list(parents_candidates.value_counts(sort=False))
sunburst_values.extend(inner_values)

candidate_values = []
for candidate in list(df_temp.index):
    candidate_values.extend([1])

sunburst_values.extend(candidate_values)

sunburst_values

[4, 4, 1, 1, 1, 1, 1, 1, 1, 1]

In [170]:
# Denne kode bruges KUN til express sunbursten
inner_names = list(parents_candidates.value_counts(sort=False).index)
inner_names.extend(parents_candidates)


colormap = []
for value in inner_names:
    colormap.append(color_dict[value])


In [171]:
data = dict(
    names=sunburst_names,
    parent=sunburst_parents,
    value=sunburst_values,
    color_map = inner_names)

fig = px.sunburst(
    data,
    names='names',
    parents='parent',
    values='value',
    branchvalues="total",
    color = "color_map",
    color_discrete_map = color_dict
    )



fig.show()

In [178]:
fig = go.Figure()

colors = ['rgb(218,241,212)','rgb(169,220,163)','rgb(15,122,55)']

data = dict(
    names=sunburst_names,
    parent=sunburst_parents,
    value=sunburst_values)

fig.add_trace(go.Sunburst(
    labels=data['names'],
    parents=data['parent'],
    values=data['value'],
    branchvalues="total",
    marker_colors = ['rgb(169,220,163)','rgb(15,122,55)'],
    insidetextorientation='radial',
    ))



fig.show()

In [188]:
parents_candidates.unique()
color_dict = {"Enig":'rgb(15,122,55)',"Delvist Enig": 'rgb(169,220,163)'}
colors_sunburst = {}
for answer in parents_candidates.unique():
    colors_sunburst[answer] = color_dict[answer]
colors_sunburst

{'Enig': 'rgb(15,122,55)', 'Delvist Enig': 'rgb(169,220,163)'}

In [191]:
df_temp =  df_nameIndex[df_nameIndex["Kommune"]==kommune]
df_temp = df_temp.sort_values(question, ascending = False)

# Parents for sunburst
value_labels = {0:"Uenig", 1:"Delvist Enig", 2:"Enig"}
parents_candidates = pd.Series([value_labels[x] for x in df_temp[question]], 
                              index =df_temp.index)

sunburst_parents = []
for answer in parents_candidates.unique():
    sunburst_parents.append("")
sunburst_parents.extend(parents_candidates)

# Names for sunburst
sunburst_names = []

inner_names = list(parents_candidates.value_counts(sort=False).index)
sunburst_names.extend(inner_names)

candidate_names = list(df_temp.index)
sunburst_names.extend(candidate_names)

# Values for sunburst
sunburst_values = []

inner_values = list(parents_candidates.value_counts(sort=False))
sunburst_values.extend(inner_values)

candidate_values = []
for candidate in list(df_temp.index):
    candidate_values.append(1)

sunburst_values.extend(candidate_values)

fig = go.Figure()

color_dict = { "Enig":'rgb(15,122,55)',"Delvist Enig": 'rgb(169,220,163)',"Uenig":'rgb(218,241,212)'}
colors_sunburst = {}
for answer in parents_candidates.unique():
    colors_sunburst[answer] = color_dict[answer]

data = dict(
    names=sunburst_names,
    parent=sunburst_parents,
    value=sunburst_values)

fig.add_trace(go.Sunburst(
    labels=data['names'],
    parents=data['parent'],
    values=data['value'],
    branchvalues="total",
    marker_colors = list(colors_sunburst.values()),
    insidetextorientation='radial',
    ))
fig.update_layout(
    autosize=False,
    width=1200,
    height=720)

